#Installing pykeen library #

In [ ]:
!pip install pykeen

In [ ]:
from pykeen.pipeline import pipeline

#Importing Packages and data#

In [ ]:
import pykeen
from pykeen.triples import TriplesFactory
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
htr = pd.read_csv("/content/drive/MyDrive/KGEMB/Head_Tail_Relationship_v2.csv")

In [ ]:
htr.head()

,Unnamed: 0,H,T,R
0,0,Toy Story,Animation,has_genre
1,1,Toy Story,Comedy,has_genre
2,2,Toy Story,Family,has_genre
3,3,Jumanji,Adventure,has_genre
4,4,Jumanji,Fantasy,has_genre


In [ ]:

htr.drop(columns=['Unnamed: 0'], inplace=True)
htr = htr[['H','R','T']]

In [ ]:
htr = htr.sample(frac=1).reset_index(drop=True)

In [ ]:
htr['T']=htr['T'].apply(lambda x: x.replace("\t",""))

In [ ]:
htr

,H,R,T
0,The Hijack That Went South,acted_by,Merja Larivaara
1,100 Years at the Movies,is_produced_by,Turner Classic Movies (TCM)
2,Friday the 13th,acted_by,Travis Van Winkle
3,Pan,has_genre,Family
4,Jesse Stone: Sea Change,acted_by,Marcia Bennett
...,...,...,...
579644,Cazuza: Time Doesn't Stop,directed_by,Walter Carvalho
579645,Narc,acted_by,Lina Giornofelice
579646,Made of Honor,acted_by,Christine Barger
579647,Dollman,has_rate,3.0


In [ ]:
htr.to_csv("/content/drive/MyDrive/KGEMB/hrt.tsv", index=False, sep='\t', header=None)

In [ ]:
hrt = pd.read_csv("/content/drive/MyDrive/KGEMB/hrt.tsv", sep="\t", header=None)

In [ ]:
hrt.head()

,0,1,2
0,Sergeant Dead Head,acted_by,Buster Keaton
1,The Black Windmill,acted_by,Michael Caine
2,Al Capone,directed_by,Richard Wilson
3,Grand Central Murder,released_in,1870-1980
4,The War of the Gargantuas,acted_by,Someshô Matsumoto


# Training RotatE Model #

In [ ]:
tf = TriplesFactory.from_path("/content/drive/MyDrive/KGEMB/hrt.tsv")

In [ ]:
training, testing = tf.split( ratios=0.9, random_state=0)

In [ ]:
training.triples

Reconstructing all label-based triples. This is expensive and rarely needed.


array([['!Women Art Revolution', 'released_in', '2001-2020'],
       ['#1 Cheerleader Camp', 'released_in', '2001-2020'],
       ['#Horror', 'released_in', '2001-2020'],
       ...,
       ['The Yearling', 'acted_by', 'Philip Seymour Hoffman'],
       ['A Nanny for Christmas', 'has_genre', 'Family'],
       ['Tough Enough', 'has_genre', 'Thriller']], dtype='<U104')

In [ ]:
result = pipeline(
    training=training,
    testing=testing,
    model='RotatE',
    optimizer='adam',
    optimizer_kwargs=dict(lr=0.001),
    training_loop='sLCWA',
    negative_sampler='basic',
    training_kwargs=dict(num_epochs=50),
    model_kwargs=dict( embedding_dim=150 )
)



No random seed is specified. Setting to 9096756.


INFO:pykeen.evaluation.evaluator:Starting batch_size search for evaluation now...


INFO:pykeen.evaluation.evaluator:Concluded batch_size search with batch_size=32.


INFO:pykeen.evaluation.evaluator:Evaluation took 388.77s seconds


In [ ]:
training

TriplesFactory(num_entities=193534, num_relations=6, num_triples=458527, inverse_triples=False, path="/content/drive/MyDrive/movies_big/kg/hrt.tsv")

In [ ]:
testing

TriplesFactory(num_entities=193534, num_relations=6, num_triples=114632, inverse_triples=False, path="/content/drive/MyDrive/movies_big/kg/hrt.tsv")

In [ ]:
result

PipelineResult(random_seed=2898692431, model=TransD(
  (loss): MarginRankingLoss()
  (regularizer): NoRegularizer()
  (entity_embeddings): Embedding(
    (_embeddings): Embedding(193534, 200)
  )
  (relation_embeddings): Embedding(
    (_embeddings): Embedding(6, 100)
  )
  (entity_projections): Embedding(
    (_embeddings): Embedding(193534, 200)
  )
  (relation_projections): Embedding(
    (_embeddings): Embedding(6, 100)
  )
), training_loop=<pykeen.training.slcwa.SLCWATrainingLoop object at 0x7f08b294da90>, losses=[0.002515676701551433, 0.0015742546718424707, 0.0012577430132681353, 0.0010570441616436935, 0.0009153402185771386, 0.0007921219518193412, 0.0007081542930233238, 0.0006359020068947385, 0.0005724804880636328, 0.0005230459601094745, 0.0004835152081431414, 0.0004479444801670782, 0.0004207730487627572, 0.00039954469677960634, 0.0003811857491634085, 0.00036453353669997255, 0.00035166798873877207, 0.00033802714043902165, 0.00033115438061861975, 0.00032163051166531916, 0.00031470

In [ ]:
result.metric_results

RankBasedMetricResults(mean_rank={'tail': {'best': 16448.11204550213, 'avg': 16448.12619512876, 'worst': 16448.14034475539}, 'both': {'best': 16097.382013748342, 'avg': 16097.395147079349, 'worst': 16097.408280410356}, 'head': {'best': 15746.651981994557, 'avg': 15746.66409902994, 'worst': 15746.676216065322}}, mean_reciprocal_rank={'tail': {'best': 0.14102422666186037, 'avg': 0.14102422548537719, 'worst': 0.14102422431620482}, 'both': {'best': 0.07485227786825153, 'avg': 0.07485227637551002, 'worst': 0.0748522748937663}, 'head': {'best': 0.008680329074642687, 'avg': 0.008680327265642855, 'worst': 0.008680325471327745}}, hits_at_k={'tail': {'best': {1: 0.08990508758461861, 3: 0.1648579803196315, 5: 0.2009561030078861, 10: 0.24103217251727266}, 'avg': {1: 0.08990508758461861, 3: 0.1648579803196315, 5: 0.2009561030078861, 10: 0.24103217251727266}, 'worst': {1: 0.08990508758461861, 3: 0.1648579803196315, 5: 0.2009561030078861, 10: 0.24103217251727266}}, 'both': {'best': {1: 0.047185777095

In [ ]:
result.losses

[0.002515676701551433,
 0.0015742546718424707,
 0.0012577430132681353,
 0.0010570441616436935,
 0.0009153402185771386,
 0.0007921219518193412,
 0.0007081542930233238,
 0.0006359020068947385,
 0.0005724804880636328,
 0.0005230459601094745,
 0.0004835152081431414,
 0.0004479444801670782,
 0.0004207730487627572,
 0.00039954469677960634,
 0.0003811857491634085,
 0.00036453353669997255,
 0.00035166798873877207,
 0.00033802714043902165,
 0.00033115438061861975,
 0.00032163051166531916,
 0.0003147097134682209,
 0.00030846639575869065,
 0.00030279368710459597,
 0.00029744932637195353,
 0.00029294394978248724,
 0.0002875404182884956,
 0.00028196145358344566,
 0.00028069010792292304,
 0.00027654865496603073,
 0.00027620979690519935]

In [ ]:
result

PipelineResult(random_seed=281979223, model=TransR(
  (loss): MarginRankingLoss()
  (regularizer): NoRegularizer()
  (entity_embeddings): Embedding(
    (_embeddings): Embedding(193534, 200)
  )
  (relation_embeddings): Embedding(
    (_embeddings): Embedding(6, 30)
  )
  (relation_projections): Embedding(
    (_embeddings): Embedding(6, 6000)
  )
), training_loop=<pykeen.training.slcwa.SLCWATrainingLoop object at 0x7f091ee46810>, losses=[0.0025650634624131774, 0.0019202234971070575, 0.0013956774831038901, 0.0009922446629951707, 0.0007601338062110223, 0.0006179939613716983, 0.0005295871437287082, 0.00046853392185147274, 0.0004299995504621427, 0.00039962625323085744, 0.0003696209320096712, 0.00035425862655684686, 0.00033616275549073766, 0.00032001603148940847, 0.0003104003585081387, 0.00029744853551136406, 0.00028824147997812675, 0.0002797957369186454, 0.0002704840859025605, 0.000265217257918972, 0.00025977055477425045, 0.0002534173887728177, 0.0002506387021877119, 0.0002443505068546427

In [ ]:
result.save_to_directory('/content/drive/MyDrive/test_model')

# Using Embeddings to predict head or tail entities#

In [ ]:
result.model.get_tail_prediction_df("Jumanji", "acted_by")

,tail_id,tail_label,score,in_training
37193,37193,Cyrus Thiedeke,-0.358402,True
153025,153025,Sarah Gilson,-0.503115,True
147609,147609,Robyn Driscoll,-0.546357,True
100683,100683,Laura Bell Bundy,-0.550779,True
91652,91652,June Lion,-0.726658,True
...,...,...,...,...
167324,167324,Tender Cousins,-3.534545,False
187893,187893,White Banners,-3.536766,False
22043,22043,Born to Dance,-3.550373,False
170464,170464,The Fearless Freaks,-3.584734,False


In [ ]:
hrt[(hrt[0]=='Jumanji')&(hrt[1]=='acted_by')]

,0,1,2
220,Jumanji,acted_by,Robin Williams
3284,Jumanji,acted_by,Malcolm Stewart
68155,Jumanji,acted_by,Gary Joseph Thorup
70109,Jumanji,acted_by,Jonathan Hyde
90470,Jumanji,acted_by,Sarah Gilson
122842,Jumanji,acted_by,Annabel Kershaw
130392,Jumanji,acted_by,Brandon Obray
157923,Jumanji,acted_by,Patricia Clarkson
201656,Jumanji,acted_by,Lloyd Berry
204010,Jumanji,acted_by,Darryl Henriques


In [ ]:
result.model.get_head_prediction_df("acted_by", "Robin Williams")

,head_id,head_label,score,in_training
159128,159128,Soapdish,-0.300229,False
173701,173701,The Pickle,-0.303013,False
126226,126226,National Lampoon's Loaded Weapon 1,-0.308623,False
42387,42387,Deconstructing Harry,-0.321400,False
155630,155630,Shadows and Fog,-0.337834,False
...,...,...,...,...
21180,21180,Bob Burns,-3.237560,False
15378,15378,Ayşen Gruda,-3.265323,False
109755,109755,Makoto Satô,-3.276598,False
170478,170478,The Field Mouse,-3.299488,False


In [ ]:
result.model.get_head_prediction_df("acted_by", "Kirsten Dunst")

,head_id,head_label,score,in_training
117814,117814,Mercury Rising,-0.393699,False
177845,177845,Timeline,-0.459096,False
59803,59803,G.I. Joe: Retaliation,-0.466196,False
173701,173701,The Pickle,-0.468517,False
139465,139465,Predators,-0.470015,False
...,...,...,...,...
33958,33958,Clay Bros. Motion Pictures,-3.327530,False
111050,111050,Marco Confortola,-3.334938,False
192391,192391,Zen Films,-3.363224,False
115096,115096,Masato Hirano,-3.386395,False


In [ ]:
set(hrt[(hrt[1]=='acted_by') & (hrt[2]=='Kirsten Dunst')][0]).intersection(result.model.get_head_prediction_df("acted_by", "Kirsten Dunst")['head_label'][:20])

set()

In [ ]:
hrt[(hrt[0]=='Anna Karenina')&(hrt[1]=='is_produced_by')]

,0,1,2
187345,Anna Karenina,is_produced_by,Working Title Films
229593,Anna Karenina,is_produced_by,Universal Pictures
269929,Anna Karenina,is_produced_by,Metro-Goldwyn-Mayer (MGM)
350733,Anna Karenina,is_produced_by,Focus Features
369488,Anna Karenina,is_produced_by,Warner Bros.
469550,Anna Karenina,is_produced_by,Studio Trite
562967,Anna Karenina,is_produced_by,Icon Entertainment International


In [ ]:
result.model.get_tail_prediction_df("Anna Karenina", "is_produced_by")

,tail_id,tail_label,score,in_training
15510,15510,BBC Films,-0.677859,False
74613,74613,Ingenious Film Partners,-0.689767,False
73567,73567,Icon Entertainment International,-0.706779,True
75778,75778,Isle of Man Film,-0.736335,False
25188,25188,Bórd Scannán na hÉireann,-0.740826,False
...,...,...,...,...
175523,175523,The Ugly Ones,-3.212108,False
54422,54422,Faces of Death II,-3.240775,False
165739,165739,Take a Girl Like You,-3.241696,False
171135,171135,The Great Spy Chase,-3.258018,False


In [ ]:
result.model.get_tail_prediction_df("Anna Karenina", "has_genre")

,tail_id,tail_label,score,in_training
46956,46956,Drama,-0.817037,True
177167,177167,Thriller,-0.878295,False
124785,124785,Mystery,-0.922960,False
148448,148448,Romance,-0.926935,False
11009,11009,Anna Karenina,-1.000000,False
...,...,...,...,...
125086,125086,Nadia Sibirskaïa,-3.446537,False
95532,95532,Ken White,-3.461803,False
115999,115999,Matthäus Bleibinger,-3.478089,False
77218,77218,Jackie Vernon,-3.482459,False


In [ ]:
result

PipelineResult(random_seed=2898692431, model=TransD(
  (loss): MarginRankingLoss()
  (regularizer): NoRegularizer()
  (entity_embeddings): Embedding(
    (_embeddings): Embedding(193534, 200)
  )
  (relation_embeddings): Embedding(
    (_embeddings): Embedding(6, 100)
  )
  (entity_projections): Embedding(
    (_embeddings): Embedding(193534, 200)
  )
  (relation_projections): Embedding(
    (_embeddings): Embedding(6, 100)
  )
), training_loop=<pykeen.training.slcwa.SLCWATrainingLoop object at 0x7f08b294da90>, losses=[0.002515676701551433, 0.0015742546718424707, 0.0012577430132681353, 0.0010570441616436935, 0.0009153402185771386, 0.0007921219518193412, 0.0007081542930233238, 0.0006359020068947385, 0.0005724804880636328, 0.0005230459601094745, 0.0004835152081431414, 0.0004479444801670782, 0.0004207730487627572, 0.00039954469677960634, 0.0003811857491634085, 0.00036453353669997255, 0.00035166798873877207, 0.00033802714043902165, 0.00033115438061861975, 0.00032163051166531916, 0.00031470

In [ ]:
result = pipeline(
    training=training,
    testing=testing,
    model='RotatE',
    training_loop='sLCWA',
    training_kwargs=dict(num_epochs=30),
    model_kwargs=dict(
       embedding_dim=200,
   )
)

INFO:pykeen.evaluation.evaluator:Starting batch_size search for evaluation now...


INFO:pykeen.evaluation.evaluator:Concluded batch_size search with batch_size=32.


INFO:pykeen.evaluation.evaluator:Evaluation took 1069.69s seconds


In [ ]:
result

PipelineResult(random_seed=3307414901, model=RotatE(
  (loss): MarginRankingLoss()
  (regularizer): NoRegularizer()
  (entity_embeddings): Embedding(
    (_embeddings): Embedding(193534, 400)
  )
  (relation_embeddings): Embedding(
    (_embeddings): Embedding(6, 400)
  )
), training_loop=<pykeen.training.slcwa.SLCWATrainingLoop object at 0x7f091eff2d50>, losses=[0.0027666916009604922, 0.0016823593237335543, 0.001352641844373931, 0.0011552393003689946, 0.0010063669931819244, 0.0008740990072076776, 0.0007606538015942634, 0.0006628644648986306, 0.000577908790369419, 0.0004971087829447198, 0.0004256772417202086, 0.00036216844611817637, 0.00030921825780466015, 0.0002649056357772662, 0.0002287940979036729, 0.0001985274661369392, 0.00017782722742122828, 0.0001647270126935673, 0.00015197102341117506, 0.00014073243480445166, 0.00013282256592960127, 0.00012512732995911146, 0.00011870895991991739, 0.00011445877131162664, 0.00010800295139565872, 0.000103798644924829, 0.00010074955117907038, 9.775

In [ ]:
result.model.get_tail_prediction_df("Jumanji", "acted_by")

,tail_id,tail_label,score,in_training
56735,56735,Florica Vlad,-4.322978,True
21827,21827,Bonnie Hunt,-4.493131,True
22787,22787,Brenda Lockmuller,-4.511735,True
100683,100683,Laura Bell Bundy,-4.551834,True
97588,97588,Kirsten Dunst,-4.588308,True
...,...,...,...,...
169525,169525,The Company She Keeps,-7.830229,False
156573,156573,Shell,-7.856206,False
187949,187949,White Rage,-7.983610,False
10704,10704,Ankhon Dekhi,-8.079608,False


In [ ]:
result.model.get_tail_prediction_df("Anna Karenina", "has_genre")

,tail_id,tail_label,score,in_training
46956,46956,Drama,-4.567401,True
148448,148448,Romance,-4.812747,False
186721,186721,War,-4.932533,False
71090,71090,History,-4.958970,False
124785,124785,Mystery,-5.043876,False
...,...,...,...,...
183252,183252,Valentina Rodríguez,-7.481876,False
102841,102841,Les Films Alyne,-7.531323,False
36882,36882,Curfew,-7.551017,False
10704,10704,Ankhon Dekhi,-7.612647,False


In [ ]:
result.model.get_head_prediction_df("acted_by", "Kirsten Dunst")

,head_id,head_label,score,in_training
189274,189274,Wimbledon,-4.289385,True
160356,160356,Spider-Man 3,-4.515783,True
103338,103338,Levity,-4.518926,True
70635,70635,High Strung,-4.519812,True
91538,91538,Jumanji,-4.588308,True
...,...,...,...,...
82367,82367,Jennifer Martin,-7.759177,False
140836,140836,Rachel Devirys,-7.775012,False
147240,147240,Robert et Raymond Hakim,-7.776433,False
158653,158653,Sky Hirschkron,-7.813247,False


In [ ]:
result.metric_results

RankBasedMetricResults(mean_rank={'tail': {'avg': 20124.381616826016, 'worst': 20124.3992515179, 'best': 20124.363982134135}, 'head': {'avg': 17195.52986077186, 'worst': 17195.543748691463, 'best': 17195.51597285226}, 'both': {'avg': 18659.95573879894, 'worst': 18659.971500104682, 'best': 18659.939977493195}}, mean_reciprocal_rank={'tail': {'avg': 0.14935642218016645, 'worst': 0.1493562230709453, 'best': 0.14935666981776843}, 'head': {'avg': 0.005692470521826179, 'worst': 0.005692469514792032, 'best': 0.0056924715344999505}, 'both': {'avg': 0.07752444635099631, 'worst': 0.07752434629286865, 'best': 0.07752457067613418}}, hits_at_k={'tail': {'avg': {1: 0.09000104682811083, 3: 0.1781701444622793, 5: 0.2217007467373857, 10: 0.2722276502198339}, 'worst': {1: 0.09000104682811083, 3: 0.1781701444622793, 5: 0.2217007467373857, 10: 0.2722276502198339}, 'best': {1: 0.09000104682811083, 3: 0.1781701444622793, 5: 0.2217007467373857, 10: 0.2722276502198339}}, 'head': {'avg': {1: 0.0028613301695861

In [ ]:
result.save_to_directory('/content/drive/MyDrive/movies_big/kg/rotate')

In [ ]:
result.model.get_tail_prediction_df("Spider-Man 3", "acted_by")

,tail_id,tail_label,score,in_training
107758,107758,Luis Rosa,-4.202146,True
179826,179826,Tony Galtieri,-4.303134,True
120040,120040,Michele-Nanette Miller,-4.336683,True
99464,99464,La Rivers,-4.414087,True
179648,179648,Toni Wynne,-4.418361,True
...,...,...,...,...
40209,40209,David Brent: Life on the Road,-7.815429,False
36882,36882,Curfew,-7.819993,False
172932,172932,The Middle of the World,-7.825438,False
173079,173079,The Most Hated Woman in America,-7.842744,False
